# Basic work

In [1]:
var clear = (() => {});

In [2]:
require('dotenv').config({ path: '../.env' });
var { evm, wallet, constants, common } = require('../utils');
var { toUnit } = require('../utils/bn');
var { getLatestBlockTimestamp } = require('../utils/evm');
var { ethers, web3, Web3, artifacts } = require('hardhat');
var moment = require('moment');
var Plot = require('plotly-notebook-js');
clear();

In [3]:
(async () => {
    await evm.reset({
        jsonRpcUrl: process.env.MAINNET_HTTPS_URL,
        blockNumber: constants.FORK_BLOCK_NUMBER,
    });
})();

In [4]:
var jobOwner, keeper;
(async () => {
    [jobOwner, keeper] = await ethers.getSigners();
})();

In [5]:
var keep3r, governance, keep3rV1, helper;
(async () => {
    const data = await common.setupKeep3r();
    keep3r = data.keep3r;
    governance = data.governance;
    keep3rV1 = data.keep3rV1;
    helper = data.helper;
})();

In [7]:
var job;
(async () => {
    job = await common.createJobForTest(keep3r.address, jobOwner);
    keep3r.connect(governance).addJob(job.address);
})();

In [8]:
var w3Keep3r;
(async () => {
    const artifact = await artifacts.readArtifact('Keep3r');
    w3Keep3r = new web3.eth.Contract(artifact.abi, keep3r.address);
})();

## Setup plot

In [9]:
var unixToDate = (unix) => {
    return moment.unix(unix).format('YYYY-MM-DD HH:mm:ss');
}

In [10]:
var precision = 10 ** 2;
var bnToNumber = (bn) => {
    return test = Number(bn.mul(precision).div(toUnit(1)).toString()) / precision;
}

In [11]:
var pendingCreditsTrace = {};
var recordPendingCredits = async (jobAddress) => {
    if (!pendingCreditsTrace[jobAddress]) pendingCreditsTrace[jobAddress] = { x: [], y: [], name: 'Pending credits' };
    
    const pendingCredits = await keep3r.jobPendingCredits(job.address);
    pendingCreditsTrace[jobAddress].x.push(unixToDate(await getLatestBlockTimestamp()));
    pendingCreditsTrace[jobAddress].y.push(bnToNumber(pendingCredits));
};

In [12]:
var currentCreditsTrace = {};
var recordCurrentCredits = async (jobAddress) => {
    if (!currentCreditsTrace[jobAddress]) currentCreditsTrace[jobAddress] = { x: [], y: [], name: 'Current credits' };
    
    const currentCredits = await keep3r.jobLiquidityCredits(job.address);
    currentCreditsTrace[jobAddress].x.push(unixToDate(await getLatestBlockTimestamp()));
    currentCreditsTrace[jobAddress].y.push(bnToNumber(currentCredits));
};

## Work, work, work

#### Liquidity setup

In [13]:
var kp3rWeth, kp3rWethWale;
(async () => {
    kp3rWethWale = await wallet.impersonate(constants.RICH_KP3R_WETH_POOL_ADDRESS);
    kp3rWeth = await ethers.getContractAt('@openzeppelin/contracts/token/ERC20/IERC20.sol:IERC20', constants.KP3R_WETH_POOL_ADDRESS);
    await keep3r.connect(governance).approveLiquidity(kp3rWeth.address);
})();

#### Keeper setup

In [14]:
(async () => {
    await keep3r.connect(keeper).bond(constants.KP3R_V1_ADDRESS, 0);
    await evm.advanceTimeAndBlock(moment.duration(3, 'days').as('seconds'));
    await keep3r.connect(keeper).activate(constants.KP3R_V1_ADDRESS);
})();

#### Add 100 LP tokens to job

In [15]:
(async () => {
    const liquidityToAdd = toUnit(1);
    await kp3rWeth.connect(kp3rWethWale).transfer(jobOwner.address, liquidityToAdd);
    await kp3rWeth.connect(jobOwner).approve(keep3r.address, liquidityToAdd);
    await keep3r.connect(jobOwner).addLiquidityToJob(job.address, kp3rWeth.address, liquidityToAdd);
})();

In [16]:
(async () => {
    await evm.advanceTimeAndBlock(moment.duration(8, 'hours').as('seconds'));
})();

#### Work for 4 periods

In [17]:
(async () => {
    const period = moment.duration(5, 'days').as('seconds');
    const timeToWork = period * 10;
    const startedToWorkAt = await getLatestBlockTimestamp();
    const maxRestHours = 72;
    
    await recordPendingCredits(job.address);
    await recordCurrentCredits(job.address);
    
    await evm.advanceTimeAndBlock(moment.duration(maxRestHours, 'hours').as('seconds'));
    
    await recordPendingCredits(job.address);
    await recordCurrentCredits(job.address);
    
    while ((await getLatestBlockTimestamp()) - startedToWorkAt < timeToWork) {
//         const workDifficulty = Math.ceil(Math.random() * 1);
//         const workRest = moment.duration(Math.ceil(Math.random() * 72), 'hours').as('seconds');
        
//         const workDifficulty = Math.ceil(Math.random() * 1);
        const workRest = moment.duration(maxRestHours, 'hours').as('seconds');
        
//         console.log(`Working with difficulty: ${workDifficulty}, sleeping ${moment.duration(workRest, 'seconds').as('hours')} hours`);
        console.log(`Sleeping ${moment.duration(workRest, 'seconds').as('hours')} hours`);
        await job.connect(keeper).work();
        await recordPendingCredits(job.address);
        await recordCurrentCredits(job.address);
        
        await evm.advanceTimeAndBlock(workRest);
        await recordPendingCredits(job.address);
        await recordCurrentCredits(job.address);
    }
})();

Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours
Sleeping 72 hours


## Draw plot

In [18]:
var plot = Plot.createPlot([], { title: 'Basic work' });
plot.addTraces([pendingCreditsTrace[job.address]]);
plot.addTraces([currentCreditsTrace[job.address]]);
$$html$$ = plot.render();